In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('tallas.csv')
data.T

In [4]:
talla_map = {'L':1, 'M':0}
data['color'] = data.talla.map(talla_map)
data.T

In [9]:
new = pd.DataFrame([[160,63,None,None]])
new.columns = data.columns
new1 = pd.DataFrame([[164,62,None,None]])
new1.columns = data.columns
plt.scatter(data.altura, data.peso, c=data.color)
plt.scatter(new.altura, new.peso, c='r')
new

In [10]:
def get_closest_point(data, point, k=5):
    data_i = data.iloc[:, 0:2].values
    point_i = point.iloc[:, 0:2].values
    talla = data.talla.values
    dist = [[i, np.linalg.norm(data_i[i] - point_i), talla[i]] for i in range(len(data))]
    dist = pd.DataFrame(dist)
    dist.columns = ['index', 'distance', 'talla']
    return dist.sort_values('distance').head(k)

In [13]:
c = get_closest_point(data, new)
c

In [43]:
def show_closest_points(data, point, cercano, color='c'):
    plt.scatter(data.altura, data.peso, c=data.color)
    plt.scatter(point.altura, point.peso, c=color)
    for c in cercano.values:
        p = data.loc[c[0], :]
        plt.plot([point.altura.values[0], p.altura], [point.peso.values[0], p.peso])
    plt.show()


In [44]:
c = get_closest_point(data, new)
c
show_closest_points(data, new, c, 'r')

In [28]:
c = get_closest_point(data, new)
c

# Recomendados de animes

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
anime = pd.read_csv('https://bit.ly/3Ye5Scj')
anime.head(2)

In [46]:
anime.shape, anime.isnull().isnull().sum()

### Data presprocessing
Eliminar simbolos raros y transformar a minúsculas

In [47]:
import re
anime.name = anime.name.map(lambda name:re.sub('[^A-Za-z0-9]+', " ", name))
anime.head(2)

In [49]:
known_animes = {"Naruto Shippuuden":500, "One Piece":784,"Detective Conan":854, "Dragon Ball Super":86,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}
for k,v in known_animes.items():
    anime.loc[anime.name==k, 'episodes'] = v

In [50]:
anime.loc[(anime.type =='OVA')&(anime.episodes== 'Unknown'), 'episodes'] = 1
anime.loc[(anime.type =='Movies')&(anime.episodes=='Unknown'), 'episodes'] = 1
anime.loc[(anime.genre =='Hentai')&(anime.episodes=='Unknown'), 'episodes'] = 1

In [52]:
anime.episodes = anime.episodes.map(lambda x: np.nan if x == 'Unknown' else x)
anime.episodes.fillna(anime.episodes.median(), inplace=True)

In [53]:
anime.members = anime.members.astype(float)
anime.rating = anime.rating.astype(float)
anime.rating.fillna(anime.rating.median(), inplace=True)

In [55]:
anime_features = pd.concat([anime.genre.str.get_dummies(sep = ' , '),
                           pd.get_dummies(anime.type), # Separa por columnas 
                           anime.rating, anime.members, anime.episodes],axis=1)
anime_features.head(2)

In [56]:
anime_features.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
anime_features_norm = mms.fit_transform(anime_features)

In [59]:
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
nbrs.fit(anime_features)

In [61]:
distances, indices = nbrs.kneighbors(anime_features)